In [29]:
%pip install kubeflow-training
%pip install hydra-core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
def train_func():
    import sys
    sys.path.append("/workspace/files/cosi/src")

    import os
    import subprocess
    os.chdir("files/cosi")
    subprocess.run(["python", "src/train.py"])
    # subprocess.run(["pwd", "."])
    # subprocess.run(["ls", "."])

    # from hydra import initialize_config_dir, compose

    # initialize_config_dir(version_base="1.2", config_dir="/files/cosi/config")
    # cfg = compose(config_name="config_train.yaml", return_hydra_config=True)

    # from train import main
    # main(cfg)

In [31]:
from kubeflow.training import TrainingClient, models

train = TrainingClient()

train.create_job(
    name="kube-pilot",
    train_func=train_func,
    num_procs_per_worker="1",
    packages_to_install=["torch", "torchaudio", "soundfile", "hydra-core", "omegaconf", "lightning", \
        "wandb", "pandas", "matplotlib", "git+https://github.com/kymatio/kymatio.git", 
        "git+https://github.com/tiianhk/timbremetrics.git"],
    num_workers=1,
    base_image="harbor.a.comp-teach.qmul.ac.uk/talos-image/c4dm/pytorch-2.8.0-cuda12.8-cudnn9-runtime-git:0.0.1",
    volumes=[models.V1Volume(name="files", persistent_volume_claim=models.V1PersistentVolumeClaimVolumeSource(claim_name="acw745-shared-fs")),
        models.V1Volume(name="data", persistent_volume_claim=models.V1PersistentVolumeClaimVolumeSource(claim_name="acw745-data"))
    ],
    volume_mounts=[models.V1VolumeMount(name="files", mount_path="/workspace/files"), models.V1VolumeMount(name="data", mount_path="/data")],
    resources_per_worker={
        "nvidia.com/a100-80g": "1"
    },
    queue_name="c4dm"
)

Queue 'c4dm' does not exist in namespace 'kf-acw745'. The job will be created but may not be managed by Kueue.


In [32]:
train.wait_for_job_conditions(name="kube-pilot", expected_conditions={"Running"})

train.get_job_logs(name="kube-pilot", verbose=True)

({'kube-pilot-master-0': "  DEPRECATION: Building 'kymatio' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'kymatio'. Discussion can be found at https://github.com/pypa/pip/issues/6334\n"},
 {'pod/kube-pilot-master-0': ['2025-09-16 13:23:46 Successfully assigned kf-acw745/kube-pilot-master-0 to gt04',
   '2025-09-16 13:23:47 AttachVolume.Attach succeeded for volume "pvc-2e6b5e3b-f364-41c5-ab4f-6ba40bf7eada" ',
   '2025-09-16 13:23:47 AttachVolume.Attach succeeded for volume "pvc-cbdfaedc-2bcc-40d0-94e7-aa6c71949ba5" ',
   '2025-09-16 13:24:00 Container image "harbor.a.comp-teach.qmul.ac.uk/talos-image/c4dm/pytorch-2.8.0-cuda12.8-cudnn9-runtime-git:0.0.1" already present on machine',
   '2025-